# WPROWADZENIE DO SZTUCZNEJ INTELIGENCJI - LABORATORIUM 3

+ AUTOR: **ŁUKASZ STANISZEWSKI**
+ NR INDEKSU: **304098**
+ ADRES E-MAIL: **lukasz.staniszewski.stud@pw.edu.pl**
+ KIERUNEK: **INFORMATYKA**
+ PRZEDMIOT: **Wprowadzenie do sztucznej inteligencji**
+ ZADANIE: **[LINK](https://apps.usos.pw.edu.pl/apps/f/xkfrztsf/lab3.pdf)**
+ TEMAT: **Implementacja uniwersalnego algorytmu minimax i zastosowanie go do gry "kółko i krzyżyk", a następnie zbadanie jego działania dla różnych maksymalnych głębokości drzewa rozbioru**

## 1. Import niezbednych modułów
+ **random** -> w przypadku algorytmu minimax, gdy ruchy są równie dobre, wybieramy losowy z nich
+ **math** -> w celu otrzymania liczby inf oraz -inf

In [1]:
import random
import math

## 2. Implementacja MiniMax
+ zadaniem jest implementacja algorytmu minimax w sposób uniwersalny, tzn. żeby działał dla każdej podanej mu gry
+ z założenia gra ta musi posiadać metody:
    + game.is_state_terminal(state) -> sprawdzająca czy stan state w grze game jest uznany za terminalny czy nie
    + game.get_successors(state) -> zwracająca listę stanów będących następnikami stanu state w grze game
    + game.grade_state(state) -> oceniająca dany stan albo za pomocą funkcji oceny albo za pomocą heurystycznej funkcji oceny
    + dodatkowo gra game musi, wywołując metodę minimax, oczekiwać na zwrócenie jej stanu, w którym komputer wykonał ruch
+ algorytm zostanie podzielony na dwie funkcje, pierwsza {minimax_make_best_move()} będzie zwracała ruch, który umożliwia zminimalizowanie oceny drogi, która będzie liczona (ocena) za pomocą drugiej funkcji: minimax()
+ algorytm minimaks w pythonie:

In [2]:
def minimax_make_best_move(game, state, depth):
    """
    :param game: game which is being played
    :param state: state of given game
    :param depth: max_depth of searching game tree
    :return: next move for oponent which is the best for him
    """
    # game should check whether actual state is terminal before executing this function, but to avoid some situations
    
    if game.is_state_terminal(state) is False:
        state_successors = game.get_successors(state) # creating game tree dynamically
        possible_scores = []    # list of scores for every successor
        for successor in state_successors:
            score = minimax(game, successor, depth - 1, True) # next is player, he will maximialize
            possible_scores.append(score)
        
        # now, when the list is full of scores, algorithm creates lists of best and equal states
        # list will have length = 1 if there is one best possible score
        # otherwise lists will be n-elemeneted if there is n best moves
        # and random.choice() will choose randomly one of successors-states 
        
        best_moves = []    # list of best moves
        best_score = math.inf  # minimax is going to minimize for oponent
        
        for index in range(len(possible_scores)):
            if possible_scores[index] < best_score: # if there is new best score, clear list and add state on 1st position
                best_moves.clear()
                best_moves.append(state_successors[index])
                best_score = possible_scores[index]
            elif possible_scores[index] == best_score: # if there is a 2nd best state with equal score, add to list of best_moves
                best_moves.append(state_successors[index])
        bestMove = random.choice(best_moves)
        return bestMove
    else:
    # AFAIK it will never happen it game, because before we start a move, we check whether actual state is terminal
        return state


def minimax(game, state, depth, is_max):
    """
    :param game: game which is being played
    :param state: state of given game
    :param depth: depth on which game tree is being searched
    :param is_max: decides whether to maximize or minimize move
    :return: value of best move
    """
    if game.is_state_terminal(state) is not False or depth <= 0:
        # grade it by w(s) if s is terminal, otherwise use heuristics
        return game.grade_state(state)
    state_successors = game.get_successors(state)
    if is_max:
        # player will maximalize
        best_val = -math.inf
        for successor in state_successors:
            new_val = minimax(game, successor, depth - 1, False)
            if new_val > best_val:
                best_val = new_val
        return best_val
    else:
        # oponent will minimalize
        best_val = math.inf
        for successor in state_successors:
            new_val = minimax(game, successor, depth-1, True)
            if new_val < best_val:
                best_val = new_val
        return best_val

## 3. Implementacja gry w kółko i krzyżyk (TicTacToe)
+ zadaniem jest zaimplementowanie gry w kółko i krzyżyk, tak aby rozgrywka miała miejsce między graczem a komputerem (którego ruchy będą sterowane minimaxem).
+ gracz powinien dowiadywać się o aktualnym stanie rozgrywki, a także wprowadzać ruchy za pomocą dowolnego interfejsu (w rozwiązaniu postanowiono posłużyć się standardowym wej/wyj - za pomocą input() oraz print())
+ dodatkowo wprowadziłem możliwość zakończenia gry przez gracza wczesniej (może będzie korzystne w przypadku testowania)
+ gra moze byc zarowno rozpoczeta przez uzytkownika (gracza) jak i oponenta (komputer sterowany minimaxem)
+ klasa TicTacToe (kolkokrzyzyk) w pythonie:

In [3]:
class TicTacToe:
    def __init__(self, max_depth=10, is_player_starting=True):
        """
        :param max_depth: maximum density of searching game tree
        """
        self.actual_state = TicTacToeState()     # empty board on the start
        self.create_board()
        self.player1 = 'X'
        self.player2 = 'O'
        self.is_finished = False                 # game is not finished
        self.max_depth = max_depth
        self.is_stopped = False                  # if player wrote "stop", game ends
        self.is_player_starting = is_player_starting  # decides whether player or oponent starts game

    def check_terminal(self, state=None):
        """
        If state is given, it checks given state, otherwise it checks game's actual state.
        Returns:
        True -> is won by somebody
        False -> is not won by anyone and there is still possible move
        None -> there is no possible move and no-one won
        """
        # checking bias
        board = self.actual_state.board if state is None else state.board
        if board[0] == board[4] == board[8] != ' ' or board[2] == board[4] == board[6] != ' ':
            if state is None:
                self.is_finished = True
            return True
        for row_col_num in range(3):
            # checking horizontal
            if board[row_col_num * 3] == board[row_col_num * 3 + 1] == board[row_col_num * 3 + 2] != ' ':
                if state is None:
                    self.is_finished = True
                return True
            # checking vertical
            if board[row_col_num] == board[row_col_num + 3] == board[row_col_num + 6] != ' ':
                if state is None:
                    self.is_finished = True
                return True
        # checking whether there is still possible move
        for character in board:
            if character == ' ':
                if state is None:
                    self.is_finished = False
                return False
        if state is None:
            self.is_finished = None
        return None

    def create_board(self):
        self.actual_state.make_empty()

    def __str__(self):
        """
        :return: stringified actual state
        """
        return "\n---|---|---\n".join(["|".join([f" {self.actual_state.board[3*row + col]} "
                                                 for col in range(3)]) for row in range(3)])

    def print_status(self):
        """
        Prints information who made move (player or oponent), how board actually looks like
        and information if game is finished.
        """
        # prints who made move
        print(f"\nPLAYER '{self.actual_state.made_move}' MADE MOVE!\n")
        # prints actual board
        print(self)
        # prints info who won or when there is a draw
        if self.is_finished is True:
            print(f"\nGAME IS FINISHED! PLAYER '{self.actual_state.made_move}' WON!\n")
        elif self.is_finished is None:
            print(f"\nGAME IS FINISHED! NO-ONE WON!\n")

    def play_game(self):
        """
        Main loop playing game. Function creates board and loop is on until game is finished.
        During loop, player and oponent are making moves for a change.
        If user writes 'stop' during game, game stops.
        """
        print(self)
        self.create_board()
        while True:
            # if player is starting, he takes move if board is empty or if computer made move
            if self.is_player_starting:
                if self.actual_state.made_move == self.player2 or self.actual_state.made_move is None:
                    self.get_player_move()
                # computer makes move only if player made move
                else:
                    self.get_minimax_move()
            # if computer is starting, he takes move if board empty or player made move
            else:
                # particular case, when board is empty, we do not want to search game tree so deep
                # because there is no player move
                if self.actual_state.made_move is None:
                    # 5 is hardcoded
                    starting_depth = 5 if self.max_depth > 5 else self.max_depth
                    self.get_minimax_move(starting_depth)
                # if player made move
                elif self.actual_state.made_move == self.player1:
                    self.get_minimax_move()
                # if computer made move
                else:
                    self.get_player_move()
            # checks whether player typed "stop" to stop program
            if self.is_stopped:
                return
            # checks whether game is finished
            self.check_terminal()
            # print actual state of game
            self.print_status()
            # breaks infinity loop if game is finished by someones win or draw
            if self.is_finished is not False:
                return

    def get_player_move(self):
        """
        Function takes an int from input, validates it and changes state of game if is ok.
        If user input 'stop' game is stopped.
        """
        # while user do not give a valid input
        while True:
            move_str = input(f"\nPlayer '{self.player1}' move [0-8]: ")
            # if user input 'stop'
            if move_str == "stop":
                print("GAME STOPPED")
                self.is_stopped = True
                return
            # if user give index on board
            move = int(move_str)
            if 0 <= move <= 8 and self.actual_state.board[move] == " ":
                self.actual_state.board[move] = self.player1
                self.actual_state.made_move = 'X'     # player = 'X'
                return

    def get_minimax_move(self, max_depth=None):
        """
        Function makes oponent move by using minimax algorithm.
        Gives to minimax function param max_depth if given else self.max_depth.
        """
        if max_depth is None:
            max_depth = self.max_depth
        self.actual_state = minimax_make_best_move(self, self.actual_state, max_depth)
        self.actual_state.made_move = 'O'      # computer = O

    def is_state_terminal(self, state):
        return self.check_terminal(state)

    def grade_state(self, state):
        """
        If state is terminal, function returns 1/0/-1.
        Else returns heuristic grade.
        """
        if self.check_terminal(state) is True:
            if state.made_move == 'X':
                state.rate = 50   # player wins
                return 50
            else:
                state.rate = -50  # oponent wins
                return -50
        elif self.check_terminal(state) is None:
            state.rate = 0       # draw
            return 0
        else:                    # state is not terminal, but depth reached 0 -> use heuristic function
            state.rate = self.heuristic_function(state)
            return self.heuristic_function(state)

    def heuristic_function(self, state):
        """
        Simple heuristic function for tic-tac-toe from lecture.
        Board of heuristic looks like:
         3 | 2 | 3
        ---|---|---
         2 | 4 | 2
        ---|---|---
         3 | 2 | 3
         Function sums up values of this board for 'X' positions and subtracts values for 'O' positions.
         I.E.:

                     O | O |
                    ---|---|---
         For board   X | X |     function returns 4 + 2 - 3 - 2 = 1
                    ---|---|---
                       |   |
        
        Before checking what is sum of values, functions checks whether given state is terminal
        and returns 50 if player wins by doing this move and -50 if player lose by doing this move.
        """
        result_of_move = self.is_state_terminal(state)
        if result_of_move is True and state.made_move == 'X':
            return 50
        elif result_of_move is True and state.made_move == 'O':
            return -50
        heuristic_sum = 0
        heuristic_board = [3, 2, 3, 2, 4, 2, 3, 2, 3]
        bad_character = 'O'
        for index in range(len(state.board)):
            if state.board[index] == state.made_move:
                heuristic_sum += heuristic_board[index]
            elif state.board[index] == bad_character:
                heuristic_sum -= heuristic_board[index]
        return heuristic_sum

    def get_successors(self, state):
        """
        :param state: state from which successors are being created

            EXAMPLE STATE:

            state.board = [
            'X', 'O', 'X',
            'X', 'O', 'O',
            ' ', 'X', ' ' ]
            state.rate = None
            state.made_move = 'X'

            CREATED SUCCESSORS FOR EXAMPLE:
            successors[0].board = [                          successors[1].board = [
            'X', 'O', 'X',                                  'X', 'O', 'X',
            'X', 'O', 'O',                                  'X', 'O', 'O',
            'O', 'X', ' ']                                  ' ', 'X', 'O']
            successors[0].rate = None                        successors[1].rate = None
            successors[0].made_move = 'O'                    successors[1].made_move = 'O'

        :return: list of states which are successors
        """
        successors = []
        for index in range(9):
            if state.board[index] == ' ':
                successors.append(state.create_successor(index, self.is_player_starting))
        return successors


class TicTacToeState:
    """
    State of TicTacToe game.
    """
    def __init__(self, board=None, made_move=None, rate=None):
        self.board = board
        self.rate = rate
        self.made_move = made_move

    def make_empty(self):
        """
        Makes game's board full of " "
        """
        self.board = [" " for _ in range(9)]

    def create_successor(self, index, is_player_starting):
        """
        Creates state's successor by changing given index, and made_move attribute.
        :param is_player_starting: to determine what to do if computer is starting and no-one made move
        :param index: place on board, which will be changed
        :return: new successor of state
        """
        if self.board[index] == ' ':
            if self.made_move is None:     # for first move in game
                if is_player_starting:
                    new_made_move = 'X'
                else:
                    new_made_move = 'O'
            else:
                new_made_move = 'O' if self.made_move == 'X' else 'X'
            new_board = self.board.copy()
            new_board[index] = new_made_move
            return TicTacToeState(new_board, new_made_move)


## 4. Schemat przeprowadzania gry
+ na początku warto ustawić ziarno dla losowania liczb pseudolosowych (ułatwia sprawdzanie kolejnych przykładowych gier)
+ następnie nalezy stworzyć obiekt reprezentujący grę w kółko i krzyżyk, jako parametr konstruktora podać maksymalną głebokość drzewa rozbioru, dodatkowo podając jako następny parametr konstruktora False, ustawiamy rozpoczynającego rozgrywkę na oponenta (domyslnie rozpoczyna user)
+ na końcu należy zawołać funkcję obiektu reprezentującego grę : play_game()

### 4.1. Przykład gry rozpoczynanej przez użytkownika

In [4]:
random.seed(100)
ttt = TicTacToe(16)
ttt.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

Player 'X' move [0-8]: 0

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X |   |   
---|---|---
   | O |   
---|---|---
   |   |   

Player 'X' move [0-8]: 2

PLAYER 'X' MADE MOVE!

 X |   | X 
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | O | X 
---|---|---
   | O |   
---|---|---
   |   |   

Player 'X' move [0-8]: 7

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
   | O |   
---|---|---
   | X |   

PLAYER 'O' MADE MOVE!

 X | O | X 
---|---|---
   | O |   
---|---|---
   | X | O 

Player 'X' move [0-8]: 3

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
 X | O |   
---|---|---
   | X | O 

PLAYER 'O' MADE MOVE!

 X | O | X 
---|---|---
 X | O |   
---|---|---
 O | X | O 

Player 'X' move [0-8]: 5

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
 X | O | X 
---|---|---
 O | X | O 

GAME IS FINISHED! NO-ONE WON!



### 4.2. Przykład gry rozpoczynanej przez oponenta

In [5]:
ttt2 = TicTacToe(9, False)
ttt2.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
   |   |   
---|---|---
   | O |   

Player 'X' move [0-8]: 0

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   | O |   

PLAYER 'O' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
 O | O |   

Player 'X' move [0-8]: 8

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
 O | O | X 

PLAYER 'O' MADE MOVE!

 X |   |   
---|---|---
   | O |   
---|---|---
 O | O | X 

Player 'X' move [0-8]: 1

PLAYER 'X' MADE MOVE!

 X | X |   
---|---|---
   | O |   
---|---|---
 O | O | X 

PLAYER 'O' MADE MOVE!

 X | X | O 
---|---|---
   | O |   
---|---|---
 O | O | X 

GAME IS FINISHED! PLAYER 'O' WON!



## 5) Działanie dla różnych maksymalnych głębokości drzewa rozbioru
+ poniżej zostanie zbadane zachowanie algorytmu dla różnych maksymalnych głębokości drzewa rozbioru
+ sprawdzenie będzie dokonywane zarówno dla przypadku gdy gracz rozpoczyna rozgrywkę jak i komputer
+ przedstawione zostaną przykładowe rozgrywk: gdy gracz gra najlepiej jak może oraz gdy popełni błąd przesądzający o porażce
+ badane maksymalne głębokości drzewa rozbioru (max_depth): 8, 5, 2, 0 (dla TicTacToe maksymalna glebokosc mozliwa to wlasnie 8) 

### 5.1. max_depth = 8

#### 5.1.1. gracz rozpoczyna:

In [6]:
ttt111 = TicTacToe(8)
ttt111.play_game()
ttt112 = TicTacToe(8)
ttt112.play_game()
ttt113 = TicTacToe(8)
ttt113.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

Player 'X' move [0-8]: 0

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X |   |   
---|---|---
   | O |   
---|---|---
   |   |   

Player 'X' move [0-8]: 1

PLAYER 'X' MADE MOVE!

 X | X |   
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | X | O 
---|---|---
   | O |   
---|---|---
   |   |   

Player 'X' move [0-8]: 6

PLAYER 'X' MADE MOVE!

 X | X | O 
---|---|---
   | O |   
---|---|---
 X |   |   

PLAYER 'O' MADE MOVE!

 X | X | O 
---|---|---
 O | O |   
---|---|---
 X |   |   

Player 'X' move [0-8]: 5

PLAYER 'X' MADE MOVE!

 X | X | O 
---|---|---
 O | O | X 
---|---|---
 X |   |   

PLAYER 'O' MADE MOVE!

 X | X | O 
---|---|---
 O | O | X 
---|---|---
 X |   | O 

Player 'X' move [0-8]: 7

PLAYER 'X' MADE MOVE!

 X | X | O 
---|---|---
 O | O | X 
---|---|---
 X | X | O 

GAME IS FINISHED! NO-ONE WON!

   |   |   
---|---|--

#### 5.1.2. oponent rozpoczyna

In [9]:
ttt121 = TicTacToe(8, False)
ttt121.play_game()
ttt122 = TicTacToe(8, False)
ttt122.play_game()
ttt123 = TicTacToe(8, False)
ttt123.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
   |   |   
---|---|---
 O |   |   

Player 'X' move [0-8]: 4

PLAYER 'X' MADE MOVE!

   |   |   
---|---|---
   | X |   
---|---|---
 O |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
   | X |   
---|---|---
 O |   | O 

Player 'X' move [0-8]: 7

PLAYER 'X' MADE MOVE!

   |   |   
---|---|---
   | X |   
---|---|---
 O | X | O 

PLAYER 'O' MADE MOVE!

   | O |   
---|---|---
   | X |   
---|---|---
 O | X | O 

Player 'X' move [0-8]: 3

PLAYER 'X' MADE MOVE!

   | O |   
---|---|---
 X | X |   
---|---|---
 O | X | O 

PLAYER 'O' MADE MOVE!

   | O |   
---|---|---
 X | X | O 
---|---|---
 O | X | O 

Player 'X' move [0-8]: 2

PLAYER 'X' MADE MOVE!

   | O | X 
---|---|---
 X | X | O 
---|---|---
 O | X | O 

PLAYER 'O' MADE MOVE!

 O | O | X 
---|---|---
 X | X | O 
---|---|---
 O | X | O 

GAME IS FINISHED! NO-ONE WON!

   |   |   
---|---|---
   |   |   
---|---|---


+ jak widać niezależnie czy rozgrywka rozpoczynana jest przez uzytkownika czy oponenta, gra konczy się albo remisem (w przypadku nie popełniania błędnych ruchów przez użytkownika) albo zwycięstwem oponenta
+ max_depth = 8 zapewnia więc, że komputer jest nie do pokonania, algorytm analizuje wtedy każdy możliwy ruch

### 5.2. max_depth = 5

#### 5.2.1 rozgrywkę rozpoczyna gracz 

In [7]:
ttt211 = TicTacToe(5)
ttt211.play_game()
ttt212 = TicTacToe(5)
ttt212.play_game()
ttt213 = TicTacToe(5)
ttt213.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

Player 'X' move [0-8]: 0

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X |   |   
---|---|---
 O |   |   
---|---|---
   |   |   

Player 'X' move [0-8]: 4

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
 O | X |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X |   |   
---|---|---
 O | X | O 
---|---|---
   |   |   

Player 'X' move [0-8]: 8

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
 O | X | O 
---|---|---
   |   | X 

GAME IS FINISHED! PLAYER 'X' WON!

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

Player 'X' move [0-8]: 1

PLAYER 'X' MADE MOVE!

   | X |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   | X |   
---|---|---
   |   |   
---|---|---
   | O |   

Player 'X' move [0-8]: 0

PLAYER 'X' MADE MOVE!

 X | X |   
---|---|---
   |   |   
---|---|---
   | O |   

PLAYER 'O' MADE MOVE!

 X | X | O 
---|--

#### 5.2.2. rozgrywkę rozpoczyna oponent 

In [8]:
ttt221 = TicTacToe(5, False)
ttt221.play_game()
ttt222 = TicTacToe(5, False)
ttt222.play_game()
ttt223 = TicTacToe(5, False)
ttt223.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
   |   | O 
---|---|---
   |   |   

Player 'X' move [0-8]: 0

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   | O 
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X |   | O 
---|---|---
   |   | O 
---|---|---
   |   |   

Player 'X' move [0-8]: 8

PLAYER 'X' MADE MOVE!

 X |   | O 
---|---|---
   |   | O 
---|---|---
   |   | X 

PLAYER 'O' MADE MOVE!

 X |   | O 
---|---|---
   | O | O 
---|---|---
   |   | X 

Player 'X' move [0-8]: 3

PLAYER 'X' MADE MOVE!

 X |   | O 
---|---|---
 X | O | O 
---|---|---
   |   | X 

PLAYER 'O' MADE MOVE!

 X |   | O 
---|---|---
 X | O | O 
---|---|---
 O |   | X 

GAME IS FINISHED! PLAYER 'O' WON!

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   |   | O 
---|---|---
   |   |   
---|---|---
   |   |   

Player 'X' move [0-8]: 0

PLAYER 'X' MADE MOVE!

 X |   | O 
---|---|---
   |   |   
---|---|

+ w tym przypadku zachowanie algorytmu jest nieco inne
+ w przypadku gdy grę rozpoczyna użytkownik, oponent popełnia błędy jeśli użytkownik będzie grał bezbłędnie, jednak jeśli użytkownik popełni błąd, oponent jest w stanie wygrać
+ w przypadku gdy grę rozpoczyna oponent, jego zachowanie jest podobne jak dla max_depth = 8, to znaczy, że przy błędach użytkownika, oponent wygrywa rozgrywkę; jednak gdy użytkownik gra bezbłędnie, gra kończy się remisem

### 5.3. max_depth = 2

#### 5.3.1. rozgrywkę rozpoczyna gracz

In [9]:
ttt311 = TicTacToe(2)
ttt311.play_game()
ttt312 = TicTacToe(2)
ttt312.play_game()
ttt313 = TicTacToe(2)
ttt313.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

Player 'X' move [0-8]: 0

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X |   |   
---|---|---
   | O |   
---|---|---
   |   |   

Player 'X' move [0-8]: 2

PLAYER 'X' MADE MOVE!

 X |   | X 
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | O | X 
---|---|---
   | O |   
---|---|---
   |   |   

Player 'X' move [0-8]: 7

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
   | O |   
---|---|---
   | X |   

PLAYER 'O' MADE MOVE!

 X | O | X 
---|---|---
   | O |   
---|---|---
 O | X |   

Player 'X' move [0-8]: 5

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
   | O | X 
---|---|---
 O | X |   

PLAYER 'O' MADE MOVE!

 X | O | X 
---|---|---
   | O | X 
---|---|---
 O | X | O 

Player 'X' move [0-8]: 3

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
 X | O | X 
---|---|---
 O | X | O 

GAME IS FINISHED! NO-ONE WON!

   |   |   
---|---|--

#### 5.3.2. rozgrywkę rozpoczyna oponent 

In [10]:
ttt321 = TicTacToe(2, False)
ttt321.play_game()
ttt322 = TicTacToe(2, False)
ttt322.play_game()
ttt323 = TicTacToe(2, False)
ttt323.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
   | O |   
---|---|---
   |   |   

Player 'X' move [0-8]: 0

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X |   | O 
---|---|---
   | O |   
---|---|---
   |   |   

Player 'X' move [0-8]: 6

PLAYER 'X' MADE MOVE!

 X |   | O 
---|---|---
   | O |   
---|---|---
 X |   |   

PLAYER 'O' MADE MOVE!

 X |   | O 
---|---|---
 O | O |   
---|---|---
 X |   |   

Player 'X' move [0-8]: 7

PLAYER 'X' MADE MOVE!

 X |   | O 
---|---|---
 O | O |   
---|---|---
 X | X |   

PLAYER 'O' MADE MOVE!

 X |   | O 
---|---|---
 O | O | O 
---|---|---
 X | X |   

GAME IS FINISHED! PLAYER 'O' WON!

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
   | O |   
---|---|---
   |   |   

Player 'X' move [0-8]: 0

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   | O |   
---|---|

+ w tym przypadku algorytm radzi sobie lekko lepiej (niż w 5.2) gdy grę rozpoczyna użytkownik, dzięki skuteczności funkcji heurystycznej

### 5.4. max_depth = 0 

#### 5.4.1. rozgrywkę rozpoczyna gracz 

In [11]:
ttt411 = TicTacToe(0)
ttt411.play_game()
ttt412 = TicTacToe(0)
ttt412.play_game()
ttt413 = TicTacToe(0)
ttt413.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

Player 'X' move [0-8]: 0

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | O |   
---|---|---
   |   |   
---|---|---
   |   |   

Player 'X' move [0-8]: 4

PLAYER 'X' MADE MOVE!

 X | O |   
---|---|---
   | X |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | O |   
---|---|---
   | X | O 
---|---|---
   |   |   

Player 'X' move [0-8]: 8

PLAYER 'X' MADE MOVE!

 X | O |   
---|---|---
   | X | O 
---|---|---
   |   | X 

GAME IS FINISHED! PLAYER 'X' WON!

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

Player 'X' move [0-8]: 0

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | O |   
---|---|---
   |   |   
---|---|---
   |   |   

Player 'X' move [0-8]: 2

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | O | X 
---|--

#### 5.4.2. rozgrywkę rozpoczyna oponent 

In [12]:
ttt421 = TicTacToe(0, False)
ttt421.play_game()
ttt422 = TicTacToe(0, False)
ttt422.play_game()
ttt423 = TicTacToe(0, False)
ttt423.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
   |   | O 
---|---|---
   |   |   

Player 'X' move [0-8]: 1

PLAYER 'X' MADE MOVE!

   | X |   
---|---|---
   |   | O 
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   | X |   
---|---|---
 O |   | O 
---|---|---
   |   |   

Player 'X' move [0-8]: 0

PLAYER 'X' MADE MOVE!

 X | X |   
---|---|---
 O |   | O 
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | X |   
---|---|---
 O | O | O 
---|---|---
   |   |   

GAME IS FINISHED! PLAYER 'O' WON!

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
   |   | O 
---|---|---
   |   |   

Player 'X' move [0-8]: 1

PLAYER 'X' MADE MOVE!

   | X |   
---|---|---
   |   | O 
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   | X |   
---|---|---
   |   | O 
---|---|---
   | O |   

Player 'X' move [0-8]: 0

PLAYER 'X' MADE MOVE!

 X | X |   
---|---|---
   |   | O 
---|---|

+ jak widać w tym przypadku, niezależnie kto rozpoczyna rozgrywkę, algorytm nie robi nic bardziej użytecznego niż wywoływanie funkcji heurystycznej w celu oceniania stanów
+ oponent chce minimalizować ocenę, więc będzie wybierał najpierw pozycje, dla których funkcja heurystyczna przyjmuje najmniejsze wartości (środki boków planszy, gdzie przyjmowana jest wartość 2)
+ wyjątek występuje w momencie kiedy zajmuje on 2 skrajne środki boków, a wolny jest środek planszy; wtedy, tak jak zakłada funkcja heurystyczna, wygrywa grę wykonując ruch na środku planszy
+ jednak żeby wygrał, należy go sprowokować do tego, w przypadku dobrej gry użytkownika, oponent nie ma szans na zwyciestwo
+ jak widać więc po powyższych symulacjach, głębokość poszukiwań drzewa rozbioru ma bardzo duże znaczenie dla poprawnego działania algorytmu MiniMax